# RAG基础操作

### 0.安装所需包

In [3]:
!pip install langchain_community 

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: D:\AI\training\20251112\code1211\.venv\Scripts\python.exe -m pip install --upgrade pip


In [2]:
!pip install langchain-classic 

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: D:\AI\training\20251112\code1211\.venv\Scripts\python.exe -m pip install --upgrade pip


### 1.导入本地数据

In [1]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader(r"./RAG_data/data1.txt",encoding='utf8')
documents = loader.load()

In [5]:
print(documents)

[Document(metadata={'source': './RAG_data/data1.txt'}, page_content='中国观点 - “十五五”规划增强中国科技板块投资理据\n发布者 匡正，中国首席投资总监，汇丰私人银行及财富管理\n2025年11月13日\n\n在科技领域，外部环境不确定性日益增加，尤其是与美国的贸易紧张局势和科技竞争，加剧了中国加快技术创新和自主可控的紧迫性。相比于“十四五”规划的建议，四中全会发布的“十五五”规划建议新增了“加强原始创新”的提法，更进一步为中国科技发展增加了突破性维度。\n细化来看，“十五五”规划的建议以“科技自立自强”为战略核心，明确将人工智能、集成电路、软件国产化、数字基础设施等前沿领域置于国家发展优先级前列，旨在通过系统性政策支持与资源配置，加速新质生产力形成，推动经济结构向更多内需导向和创新驱动转型。我们认为除了中国科网龙头以外，人工智能硬件、软件、游戏、以及消费电子与智能设备等细分赛道在更明确的政策利好下，盈利增长与估值都有上行空间。\n基本面也带来助力。第三季度盈利势头表明，信息科技板块的盈利同比增长30.8%，仅次于原材料板块。自2025年7月以来，中国成长行业的2026年盈利一致预期上修势头更加强劲。市场一致预期中证800成分股的2026年预测盈利同比增长10.3%，其中盈利增速最高的板块为信息科技（31.4%），主要受益于云服务提供商的资本支出上行周期，以及人工智能渗透率的加速提升。\n在充裕的市场流动性以及依旧较合理的估值支持下，我们继续看好代表中国创新潜力的科技板块，尤其是上述潜在受益于政策的细分赛道。我们运用科技加红利的杠铃策略，对冲潜在的外部不确定性和周期数据下行风险。')]


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

### Embedding

In [7]:
from langchain_openai import OpenAIEmbeddings
import os

os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-9OiJkk755GLEpfNU41E93135B6634035B3F1A98e0dC1Cb60'

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
text = '汇丰银行'
res = embeddings.embed_query(text)

In [7]:
print(res)

[-0.04082048684358597, -0.07336057722568512, 0.03728503733873367, 0.01130413543432951, -0.011868178844451904, -0.03556383028626442, -0.024934222921729088, 0.09945777803659439, -0.001251653884537518, -0.004326273687183857, 0.0025469192769378424, -0.010629608295857906, -0.003238890552893281, -0.030121101066470146, 0.026004161685705185, 0.04030877724289894, -0.03705244138836861, -0.0168050155043602, -0.006884823087602854, -0.030283916741609573, -0.018723927438259125, 0.018363405019044876, 0.018851853907108307, -0.009972526691854, -0.023550281301140785, 0.012176367454230785, -0.0028638304211199284, -0.0054456391371786594, 0.0011702454648911953, -0.03672681003808975, 0.05452035367488861, -0.037820007652044296, 0.053031742572784424, 0.009082849137485027, -0.028353376314044, 0.029865246266126633, -0.0029510536696761847, 0.024678368121385574, 0.03330765664577484, 0.031888823956251144, -0.004346625879406929, 0.01973571628332138, 0.0031749268528074026, -0.005942811723798513, 0.048612430691719055

In [8]:
len(res)

1536

### 5. 文件分割

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20, separators=["\n\n","\n", "!", "?", "。", "！", "？"],keep_separator=False )
splits = text_splitter.split_documents(documents)

In [10]:
print(splits)

[Document(metadata={'source': './RAG_data/data1.txt'}, page_content='中国观点 - “十五五”规划增强中国科技板块投资理据\n发布者 匡正，中国首席投资总监，汇丰私人银行及财富管理\n2025年11月13日'), Document(metadata={'source': './RAG_data/data1.txt'}, page_content='在科技领域，外部环境不确定性日益增加，尤其是与美国的贸易紧张局势和科技竞争，加剧了中国加快技术创新和自主可控的紧迫性。相比于“十四五”规划的建议，四中全会发布的“十五五”规划建议新增了“加强原始创新”的提法，更进一步为中国科技发展增加了突破性维度。'), Document(metadata={'source': './RAG_data/data1.txt'}, page_content='细化来看，“十五五”规划的建议以“科技自立自强”为战略核心，明确将人工智能、集成电路、软件国产化、数字基础设施等前沿领域置于国家发展优先级前列，旨在通过系统性政策支持与资源配置，加速新质生产力形成，推动经济结构向更多内需导向和创新驱动转型。我们认为除了中国科网龙头以外，人工智能硬件、软件、游戏、以及消费电子与智能设备等细分赛道在更明确的政策利好下，盈利增长与估值都有上行空间。'), Document(metadata={'source': './RAG_data/data1.txt'}, page_content='基本面也带来助力。第三季度盈利势头表明，信息科技板块的盈利同比增长30.8%，仅次于原材料板块。自2025年7月以来，中国成长行业的2026年盈利一致预期上修势头更加强劲。市场一致预期中证800成分股的2026年预测盈利同比增长10.3%，其中盈利增速最高的板块为信息科技（31.4%），主要受益于云服务提供商的资本支出上行周期，以及人工智能渗透率的加速提升。'), Document(metadata={'source': './RAG_data/data1.txt'}, page_content='在充裕的市场流动性以及依旧较合理的估值支持下，我们继续看好代表中国创新潜力的科技板块，尤其是上述潜在受益于政策的细分赛道。我们运用科技加红利的杠铃策略，对冲潜在的外

In [11]:
len(splits)

5

### 6. 存储到Chroma数据库中

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [13]:
ids = vector_store.add_documents(documents=splits)

In [14]:
print(ids)

['c5d2f9d4-8838-41a9-ac89-38781df4618c', 'e2e63d94-0ab5-4a36-8e5a-090603281c92', '8c6e3aad-5115-48d5-9db4-3b960aa77155', '646f2b84-4c69-4167-a23a-ede7d206a404', '214b8202-0c16-472f-a083-7400bf96e846']


### 7. 构建检索器

In [15]:
retriever = vector_store.as_retriever()

### 8. 测试检索器

In [16]:
retrieved_docs = retriever.invoke("预期盈利")

len(retrieved_docs)

4

In [17]:
print(retrieved_docs[0].page_content)

基本面也带来助力。第三季度盈利势头表明，信息科技板块的盈利同比增长30.8%，仅次于原材料板块。自2025年7月以来，中国成长行业的2026年盈利一致预期上修势头更加强劲。市场一致预期中证800成分股的2026年预测盈利同比增长10.3%，其中盈利增速最高的板块为信息科技（31.4%），主要受益于云服务提供商的资本支出上行周期，以及人工智能渗透率的加速提升。


In [18]:
print(retrieved_docs[1])

page_content='在充裕的市场流动性以及依旧较合理的估值支持下，我们继续看好代表中国创新潜力的科技板块，尤其是上述潜在受益于政策的细分赛道。我们运用科技加红利的杠铃策略，对冲潜在的外部不确定性和周期数据下行风险。' metadata={'source': './RAG_data/data1.txt'}


### 9.构建检索链

In [19]:
import os
os.environ["DEEPSEEK_API_BASE"] = 'https://api.deepseek.com/v1'
os.environ["DEEPSEEK_API_KEY"] = 'sk-60e9910f323c46c3aeb645951b2d45ec'

In [20]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [21]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [23]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [24]:
response = rag_chain.invoke({"input": "中国股市信息科技板块2026年预期盈利？"})
response["answer"]

'根据您提供的资料，中国股市信息科技板块在2026年的预期盈利同比增长为**31.4%**。这一增速在中证800成分股各板块中为最高。\n\n主要驱动因素包括：\n1.  云服务提供商的资本支出上行周期。\n2.  人工智能渗透率的加速提升。'

### 10.与原模型结果对比

In [25]:
from langchain_core.messages import AIMessage, HumanMessage
messages = [
    HumanMessage(content="中国股市信息科技板块2026年预期盈利？"),
]
response = llm.invoke(messages)
print(response.content)

这是一个非常重要且复杂的问题。需要明确指出的是，**没有任何机构或个人能够准确预测中国股市特定板块在两年后的盈利情况**。任何给出的具体数字都只是基于当前信息的推测和模型估算，存在极大的不确定性。

不过，我可以为您提供一个**专业的分析框架和当前市场的主流预期**，帮助您理解影响2026年信息科技板块盈利的关键因素和展望方向。

### 核心结论（基于当前共识）
综合多家主流券商（如中金、中信、华泰等）的2024-2025年展望报告，市场对信息科技板块的**中长期（未来2-3年）增长前景普遍持“结构性乐观”态度**。预期增长将主要来自以下几个高景气赛道，而非板块整体普涨。

### 影响2026年盈利的关键变量
1.  **宏观经济环境**：
    *   **国内经济复苏力度**：企业IT支出、消费者电子消费意愿与GDP增长密切相关。
    *   **全球需求**：出口导向的硬件、半导体公司受全球（尤其是欧美）经济周期影响。

2.  **政策驱动**：
    *   **“新质生产力”与数字经济**：这是国家顶层战略，对人工智能、算力基础设施、工业软件、数据要素等领域是长期利好。
    *   **自主可控与信创**：在外部科技限制背景下，国产替代（半导体、CPU、操作系统、数据库等）将持续获得政策与资金支持，相关企业营收确定性较高。
    *   **产业政策**：如对集成电路、新能源汽车、人工智能等具体行业的扶持政策。

3.  **技术周期与创新**：
    *   **人工智能的落地与应用**：这是未来2-3年最大的增长引擎。关注：
        *   **算力层**：AI芯片（GPU/ASIC）、服务器、光模块、数据中心。
        *   **应用层**：AI在金融、医疗、教育、办公、自动驾驶等领域的商业化变现能力。**2026年的盈利很大程度上取决于AI应用能否产生规模性利润**。
    *   **消费电子创新周期**：下一代智能终端（如AI PC、AI手机、XR设备）是否能在2025-2026年引爆换机潮。
    *   **半导体周期**：预计2024-2025年全球半导体行业将步入复苏周期，2026年可能处于景气阶段。

4.  **行业细分赛道预期（盈利增长点）**：
    *   **AI与算力**：**